# Equal Weighted Like Portfolio - comparisons

This example compares the performance of an equal weighted portfolio relative to several risk-based optimal portfolios with similar risk profile. 

The CVaR, SMCR, MAD, LSSD, BTAD, BTSD, GINI, SD and MV risk-based portfolios are considered. We use the `rtype=InvNrisk` optimization flag (the optimal portfolio with the same risk as the equal weights portfolio).

The results are presented below. We concluded that, at least for the data in this example, there is no clear-cut preference for a portfolio construction (with a superior performance under all market conditions).


We start by importing **azapy** and other useful packages.

In [29]:
import sys
sys.path.append("..")
import azapy as az

print(f"azapy version {az.version()}")

azapy version 1.1.1


### Collect historical market data

- `symb` is the list of stock symbols (portfolio components).
- `sdate` and `edate` are the start and end dates of historical time-series.
- `mktdir` is the name of the directory used as a buffer for market data collected from the data provider (in this case _alphavantage_).
    
> Note: if the flag `force=False` then a reading from `dir=mktdir` is attempted. If it fails, then the data provider servers will be accessed. The new data will be saved to the `dir=mktdir`. For more information see the readMkT documentation https://azapy.readthedocs.io/en/latest/.

In [30]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

sdate = "2012-01-01"
edate = "2021-07-27"

mktdir = "../MkTdata"

mktdata = az.readMkT(symb, sdate=sdate, edate=edate, file_dir=mktdir)

read GLD data from file
read TLT data from file
read XLV data from file
read VGT data from file
read PSJ data from file

Request between 2012-01-01 : 2021-07-27
                    GLD         TLT         XLV         VGT         PSJ
source            yahoo       yahoo       yahoo       yahoo       yahoo
force             False       False       False       False       False
save               True        True        True        True        True
file_dir     ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata
file_format         csv         csv         csv         csv         csv
api_key            None        None        None        None        None
verbose            True        True        True        True        True
error                No          No          No          No          No
nrow               2407        2407        2407        2407        2407
sdate        2012-01-03  2012-01-03  2012-01-03  2012-01-03  2012-01-03
edate        2021-07-27  2021-07-27  2021-07-27

### Set scenarios model parameters

Since we plan to run many scenarios, it is useful to organize their model parameters in a dictionary.
The keys are the portfolio names, and the values are the model parameters organized also as dictionaries:
- 'type' : is the portfolio class used in the simulation,
- 'm_param' : is a dictionary with parameters for `set_model` functions.

Note that:
- `'P_N'` is the reference equal weighted portfolio,
- `'P_CVaR'` is the mCVaR optimal portfolio with confidence levels `alpha_CVaR` equal weighted,
- `'P_SMCR'` is the mSMCR optimal portfolio with confidence levels `alpha_SMCR` equal weighted,
- `'P_MAD'` is the mMAD optimal portfolio with weights `coef_MAD`,
- `'P_LSD'` is the mLSD optimal portfolio with weights `coef_LSD`,
- `'P_BTAD'` is the mBTAD optimal portfolio with threshold levels `alpha_BTAD` equal weighted,
- `'P_BTSD'` is the mBTSD optimal portfolio with threshold levels `alpha_BTSD` equal weighted,
- `'P_MV'` is the MV (mean-variance) optimal portfolio,
- `'P_SD'` is the SD optimal portfolio.

- In all cases the optimization type is set to `rtype='InvNRisk'` (optimal portfolio with the same risk profile as the equal weighted portfolio)
- The `'hlength'` parameter (the length of historical data used in weights calibration) is set to 3.25 years except for `'P_GINI'` and `P_SMGINI` where it is set to 1.25 years (for computational speed convenience).

> In this case P_MV and P_SD are the same. 

In [31]:
alpha_CVaR = [0.95, 0.90, 0.85]
alpha_SMCR = [0.9, 0.80]
rtype = 'InvNrisk'
coef_MAD = [1./3.] * 3
coef_LSD = [1./3.] * 3
alpha_BTAD = [-0.01, 0]
alpha_BTSD = [-0.01, 0]
detrended = True
hlength = 3.25
verbose = True

models = {'P_CVaR': {'type': 'Port_CVaR', 'm_param': {'alpha': alpha_CVaR, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_SMCR': {'type': 'Port_SMCR', 'm_param': {'alpha': alpha_SMCR, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_MAD': {'type': 'Port_MAD', 'm_param': {'coef': coef_MAD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_LSD': {'type': 'Port_LSD', 'm_param': {'coef': coef_LSD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_BTAD': {'type': 'Port_BTAD', 'm_param': {'alpha': alpha_BTAD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose, 'detrended': True}},
          'P_BTSD': {'type': 'Port_BTSD', 'm_param': {'alpha': alpha_BTSD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose, 'detrended': True}},
          'P_GINI': {'type': 'Port_GINI', 'm_param': {'rtype': rtype, 'hlength': 1.25, 'verbose': verbose}},
          'P_MV': {'type': 'Port_MV', 'm_param': {'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_SD': {'type': 'Port_SD', 'm_param': {'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}

### Main computation loop

- `port` is a list containing the simulated time-series. We will use it to setup a simple portfolio *(see the documentation for `Port_Simple` class)*. It is a very convenient way to facilitate the visual and numerical comparisons between these portfolio performances. 
- `pp` is a dictionary holding the portfolio objects. They may be used later for further analytical inquires.  

In [32]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

InvNrisk for P_GINI on 2015-03-25 00:00:00, defaulted to single portfolio [TLT]
InvNrisk for P_GINI on 2017-03-27 00:00:00, defaulted to single portfolio [VGT]
InvNrisk for P_GINI on 2017-06-26 00:00:00, defaulted to single portfolio [VGT]
InvNrisk for P_GINI on 2017-09-25 00:00:00, defaulted to single portfolio [VGT]


In [33]:
pp['P_GINI'].pname

'P_GINI'

### Prepare the results for comparisons 

Build a `Port_Simple` class holding all the computed portfolios as components. We use this structure to build comparative graphical and numerical performance reports. The aggregated portfolio of portfolios time-series will be neglected.

>Note the call to `set_model` method that is a must.

>Observation: `Port_Simple` is the class that supports the back testing of "Buy and Hold" portfolio (_see its documentation_).
It also can be used as a tool to compare the performance of multiple portfolios. Here we use it in this latter capacity.

In [34]:
ps = az.Port_Simple(port, col='close', pname='ALL')
_ = ps.set_model()

### Time-series visualization 

We used the flag `componly=True` to plot only the portfolio components. 

In [35]:
_ = ps.port_view_all(componly=True, fancy=True, title="Relative performance")

### Portfolio performances

Note that, at least for the duration of this simulation, the P_BTAD portfolio has a similar rate of returns as 
P_N. However, P_BTAD has higher maximum drawdown than P_N. P_N has the smallest maximum drawdown among all portfolios. 

> Note: in this case P_MV anf P_SD portfolios are the same.

In [36]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
P_N,14.24,-17.43,0.816747,2020-03-18,2020-02-19,2020-04-24
P_BTAD,12.88,-18.67,0.689682,2020-03-20,2020-02-19,2020-05-20
P_GINI,12.85,-18.77,0.684649,2016-12-14,2016-07-08,2017-09-11
P_MAD,12.63,-19.70,0.640994,2020-03-20,2020-02-19,2020-05-29
P_CVaR,12.55,-20.08,0.625103,2020-03-20,2020-02-19,2020-05-18
P_LSD,12.40,-20.03,0.618811,2020-03-20,2020-02-19,2020-05-20
P_BTSD,12.60,-20.51,0.614352,2020-03-20,2020-02-19,2020-05-29
P_MV,12.16,-19.92,0.610401,2020-03-20,2020-02-19,2020-05-29
P_SD,12.16,-19.92,0.610401,2020-03-20,2020-02-19,2020-05-29


### Annual returns

Remarks:

- 2015 (only half year in this simulation) and 2016, P_N is the best performer.
- 2016 P_N is the best while P_GINI is the worst.
- 2017 P_N is the worst performer. The best performers are P_GINI.
- 2018 P_GINI has the best performance. P_N is second.
- 2019 The best performers are P_BTAD closely follow by P_CVaR, P_MAD and P_LSD. P_GINI and P_N are the worst.
- 2020 The best performers are P_GINI follow by P_N.
- 2021 (first half of the year in this simulation) P_GINI is the leader followed by P_N and P_MV (P_SD is the same with P_MV).

Conclusion: Depending on the historical period P_N may be an outperformer or an underperformer. Moreover, similar remarks can be made about any other portfolio strategy.

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio of portfolios.


In [37]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_BTAD,P_BTSD,P_CVaR,P_GINI,P_LSD,P_MAD,P_MV,P_N,P_SD,P_SMCR
year,,,,,,,,,,
2015,-2.85%,-3.80%,-3.68%,-5.92%,-3.72%,-3.39%,-2.61%,-1.59%,-2.61%,-5.70%
2016,6.35%,5.59%,3.38%,-5.23%,4.01%,6.15%,5.93%,8.21%,5.93%,2.58%
2017,28.26%,26.72%,25.98%,35.38%,26.04%,27.25%,26.88%,23.12%,26.88%,25.04%
2018,-1.88%,-0.98%,-0.23%,3.78%,-0.45%,-2.04%,-0.51%,2.52%,-0.51%,-0.43%
2019,27.33%,29.29%,29.63%,20.30%,29.26%,29.36%,27.03%,25.77%,27.03%,28.89%
2020,19.49%,20.44%,23.13%,30.91%,21.97%,19.41%,15.76%,25.54%,15.76%,20.89%
2021,4.92%,2.84%,1.74%,4.96%,1.81%,3.50%,4.79%,4.90%,4.79%,2.22%


### Monthly returns

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio. 

In [38]:
ps.port_monthly_returns(withcomp=True, componly=True, fancy=True)

### A closer look at P_BTAD portfolio

Monthly returns: are volatile.

In [39]:
pp['P_BTAD'].port_monthly_returns(fancy=True)

year,2015,2016,2017,2018,2019,2020,2021
month,,,,,,,
1,nan%,-1.50%,2.48%,3.51%,5.97%,1.54%,-0.72%
2,nan%,1.68%,3.57%,-0.44%,3.65%,-3.70%,-2.43%
3,nan%,2.85%,0.64%,-0.34%,-1.04%,-6.78%,-4.44%
4,nan%,-0.84%,1.94%,0.55%,1.53%,11.29%,1.74%
5,nan%,1.65%,3.24%,4.20%,-2.89%,5.78%,-0.53%
6,-1.55%,2.73%,-1.99%,-3.22%,7.57%,-0.12%,4.56%
7,1.87%,3.98%,3.05%,0.30%,1.87%,5.52%,2.75%
8,-5.83%,-0.70%,2.61%,5.61%,0.45%,1.13%,nan%
9,-2.34%,0.84%,2.02%,-0.93%,-0.79%,-4.46%,nan%


### P_BTAD performance

An example of individual portfolio performance inquiry. 

In [40]:
pp['P_BTAD'].port_perf(fancy=True)

,RR,DD,Beta,DD_date,DD_start,DD_end
symbol,,,,,,
P_BTAD,12.88,-18.67,0.689682,2020-03-20,2020-02-19,2020-05-20
VGT,23.10,-31.84,0.725512,2020-03-23,2020-02-19,2020-06-09
PSJ,22.12,-30.69,0.720991,2020-03-16,2020-02-19,2020-05-22
XLV,16.74,-28.40,0.589180,2020-03-23,2020-01-22,2020-07-15
TLT,4.89,-21.34,0.229103,2021-03-18,2020-08-04,NaN
GLD,0.81,-42.11,0.019280,2015-12-17,2012-10-04,2020-07-22


### P_BTAD drawdowns

Remarks: The first 2 largest drawdowns are similar. However, the 2018 drawdown was caused by the unexpected Feds rate increase while the 2020 event is the Covid-19 pandemic crash. 

In [41]:
pp['P_BTAD'].port_drawdown(fancy=True)

,DD,Date,Start,End
No,,,,
1,-18.67,2020-03-20,2020-02-19,2020-05-20
2,-15.28,2018-12-24,2018-08-31,2019-03-18
3,-11.35,2015-09-28,2015-08-05,2016-04-06
4,-10.71,2021-03-08,2021-02-11,2021-07-07
5,-8.20,2018-02-08,2018-01-26,2018-03-09


### P_BTAD weights

Remarks: At present the P_BTAD portfolio is dominated by GLD and VGT.

In [42]:
pp['P_BTAD'].get_weights(fancy=True)

,Droll,Dfix,Dhist,GLD,PSJ,TLT,VGT,XLV
0,2015-06-25,2015-06-24,2012-03-23,8.41,0.00,17.09,0.00,74.50
1,2015-09-25,2015-09-24,2012-06-22,18.55,0.00,13.04,10.97,57.44
2,2015-12-28,2015-12-24,2012-09-24,12.64,5.87,23.06,40.18,18.25
3,2016-03-28,2016-03-24,2012-12-24,13.53,2.78,27.73,32.04,23.92
4,2016-06-27,2016-06-24,2013-03-22,8.64,0.00,29.06,34.24,28.06
5,2016-09-27,2016-09-26,2013-06-26,0.00,0.00,36.07,26.12,37.81
6,2016-12-27,2016-12-23,2013-09-23,0.00,0.00,32.29,67.71,0.00
7,2017-03-28,2017-03-27,2013-12-27,0.00,0.00,33.57,66.43,0.00
8,2017-06-27,2017-06-26,2014-03-26,0.00,0.00,36.18,63.82,0.00
9,2017-09-26,2017-09-25,2014-06-25,0.00,0.00,28.01,71.99,0.00
